# PuLP で組み合わせ問題 (Set Partitioning Problem) を解く  

PuLP は Python で線形計画 (Linear Programing) 問題をモデル化し、ソルバーで解くためのツールです。  

- [coin-or/pulp: A python Linear Programming API](https://github.com/coin-or/pulp)  
- [Optimization with PuLP](https://coin-or.github.io/pulp/)  
- [pulp: Pulp classes](https://coin-or.github.io/pulp/technical/pulp.html)  

Anaconda Cloud ではパッケージは公開されていませんので、`pip install pulp` でインストールしてください。  

## 問題  

あるパーティの席割を検討しています。条件は、

- 参加者数は 18 名  
- テーブルは 5 つ  
- 各テーブルに参加者を 1 名以上、4 名以下を割り当てる  

です。この条件下で、各テーブルの幸福度の総和を最大化したいと考えています。幸福度は、    

- 参加者同士は 0 以上 1 未満の相性度というパラメータを持っている (大きいほど相性がよい)  
- 各テーブルに割り当てられた参加者同士の最も低い相性度をそのテーブルの幸福度とする

で求められるものとします。  

In [ ]:
# 必要なモジュールのインポート
import pulp
from pulp import LpProblem, LpMaximize, LpVariable, \
                 LpInteger, lpSum
import numpy as np
import pandas as pd
import itertools

In [ ]:
# 乱数のシード値を指定
np.random.seed(0)

In [ ]:
max_tables = 5
max_table_size = 4
guests = "A B C D E F G H I J K L M N O P Q R".split()
num_guests = len(guests)
print("出席者数: {} 人".format(num_guests))

In [ ]:
# 各個人間の相性度を作成する
# 対角成分が 0 で、それ以外は 0 以上 1 未満の値の対称行列とする
a = np.round(np.random.rand(num_guests**2), 4).reshape(num_guests, -1)
a_tri = np.triu(a)  # 上側に値を持つ三角行列
affinity = a_tri + a_tri.T - 2*np.diag(a_tri.diagonal())

In [ ]:
pd.DataFrame(affinity, index=guests, columns=guests)

In [ ]:
# ひとつのテーブルに対して、起こりうるすべての席割のリストを作成
possible_tables = [tuple(c) for c in pulp.allcombinations(guests, max_table_size)]
print("{} 通りの組み合わせが存在".format(len(possible_tables)))

In [ ]:
# 最後の要素を確認
possible_tables[-1]

In [ ]:
# テーブルの幸福度を求める関数
def happiness(guests: list, table: tuple, aff: np.ndarray) -> float: 
    """
    テーブルの幸福度を割り当てられた人のリストと相性度テーブルから取得する関数。
    割り当てられた人から 2 人選んで最も低い相性度がそのテーブルの幸福度となる。
    """
    if len(table) <= 1: return 0  # 簡易な if 式の書き方
    
    ret = 1
    for c in itertools.combinations(table, 2):
        a = aff[guests.index(c[0]), guests.index(c[1])]
        if ret > a:
            ret = a            
    return ret

In [ ]:
# 関数のテスト
happiness(guests, ("A", "B", "C"), affinity)

In [ ]:
# モデルの作成 目的関数を最大化する
zaseki = pulp.LpProblem(name="座席割", sense=pulp.LpMaximize)

In [ ]:
# その席割を使用するかどうかの 2 値 (0 または 1) の決定変数を作成
x = pulp.LpVariable.dicts(name="table", indexs=possible_tables, 
                          lowBound=0, upBound=1, cat=pulp.LpInteger)
len(x)

In [ ]:
# 目的関数をモデルに追加
zaseki += lpSum([happiness(guests, table, affinity) * x[table] for table in possible_tables])

In [ ]:
# 制約条件をモデルに追加

# テーブル数の制限
zaseki += lpSum([x[table] for table in possible_tables]) <= max_tables, "Maximum_number_of_tables"

# 1 人の出席者は必ずどれかひとつのテーブルに座らなければならない
for guest in guests:
    zaseki += lpSum([x[table] for table in possible_tables 
                     if guest in table]) == 1, "Must_seat_{}".format(guest)

In [ ]:
%%time 
zaseki.solve()

In [ ]:
# 結果の確認
total_happiness = 0
for table in possible_tables:
    if pulp.value(x[table]) == 1.0:
        h = happiness(guests, table, affinity)
        print(table, h)
        total_happiness += h

print("-"*20, "\n幸福度の合計: {}".format(total_happiness))